# Validation Generations (10 epochs)


In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Reload model in FP16 and merge it with LoRA weights
model_name = "meta-llama/Llama-2-7b-hf"
new_model = "/content/drive/MyDrive/Deep Learning Final Project/LIama2/Validation Models/10_epochs/checkpoint-440"
device_map = {"": 0}

# compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=use_4bit,
#     bnb_4bit_quant_type=bnb_4bit_quant_type,
#     bnb_4bit_compute_dtype=compute_dtype,
#     bnb_4bit_use_double_quant=use_nested_quant,
# )

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

(…)a/Llama-2-7b-hf/resolve/main/config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)b-hf/resolve/main/generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

(…)7b-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)lama-2-7b-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500, temperature=1.5)
result = pipe(f"When given a song title, you will write a song in the style of Taylor Swift.\nTitle: Invisible\nLyrics: She can't see the way your eyes\nLight up when you smile\nShe'll never notice how you stop and stare\nWhenever she walks by\nAnd you can't see me wanting you the way you want her\nBut you are everything to me\nAnd I just wanna show you\nShe don't even know you\nShe's never gonna love you like I want to\nAnd you just see right through me\nBut if you only knew me\nWe could be a beautiful miracle, unbelievable\nInstead of just invisible\nOh, yeah, oh\n")
print(result[0]['generated_text'])

When given a song title, you will write a song in the style of Taylor Swift.
Title: Invisible
Lyrics: She can't see the way your eyes
Light up when you smile
She'll never notice how you stop and stare
Whenever she walks by
And you can't see me wanting you the way you want her
But you are everything to me
And I just wanna show you
She don't even know you
She's never gonna love you like I want to
And you just see right through me
But if you only knew me
We could be a beautiful miracle, unbelievable
Instead of just invisible
Oh, yeah, oh
She might catch your stare and flatter your ego
But she'll never keep you down in your bed at night
All Lights Down Low, oh-oh-oh
'Cause it's dark 'til you leave
Sneak out before she wakes up next to someone else
If only she could see how he really sees her
But she has her bad days too
And you can't see me wanting you the way you want her
But you are everything to me
And I just wanna show you
She don't even know you
She's never gonna love you like I want 

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500, temperature=1.5)
result = pipe(f"When given a song title, you will write a song in the style of Taylor Swift.\nTitle: Come In with the Rain (Taylor's Version)\nLyrics: I could go back to every laugh\nBut I don't wanna go there anymore\nAnd I know all the steps up to your door\nBut I don't wanna go there anymore\nTalk to the wind, talk to the sky\nTalk to the man with the reasons why\nAnd let me know what you find\nI'll leave my window open\n'Cause I'm too tired at night to call your name\nJust know I'm right here hoping\nThat you'll come in with the rain\n")
print(result[0]['generated_text'])


When given a song title, you will write a song in the style of Taylor Swift.
Title: Come In with the Rain (Taylor's Version)
Lyrics: I could go back to every laugh
But I don't wanna go there anymore
And I know all the steps up to your door
But I don't wanna go there anymore
Talk to the wind, talk to the sky
Talk to the man with the reasons why
And let me know what you find
I'll leave my window open
'Cause I'm too tired at night to call your name
Just know I'm right here hoping
That you'll come in with the rain
Sounds of the rain bring you to me
I am standing all alone
In the room, just waitin' for you
In the silence by myself, wishin' for you
Need you now like I needed you back then
It don't seem right, but I almost believe
This dream like I used to believe in love
Comes in with the rain
This old room is waitin' for you
The silent telephones ringing for you
Need you now like I needed you back then
'Cause the dream like I used to believe in love
Comes in with the rain
It brings back my 

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700, temperature=1.5)
result = pipe(f"When given a song title, you will write a song in the style of Taylor Swift.\nTitle: When Emma Falls in Love\nLyrics: When Emma falls in love, she paces the floor\nCloses the blinds and locks the door\nWhen Emma falls in love, she calls up her mom\nJokes about the ways that this one could go wrong\nShe waits and takes her time\n'Cause Little Miss Sunshine always thinks it's gonna rain\nWhen Emma falls in love, I know\nThat boy will never be the same\n'Cause she's the kind of book that you can't put down\nLike if Cleopatra grew up in a small town\nAnd all the bad boys would be good boys\nIf they only had a chance to love her\nAnd to tell you the truth, sometimes I wish I was her\nWhen Emma falls in love, it's all on her face\nHangs in the air like stars in outer space\nWhen Emma falls in love, she disappears\nAnd we all just laugh after seein' it all these years\nWhen Emma falls apart, it's when she's alone\nShe takes on the pain and bears it on her own\n'Cause when Emma falls in love, she's in it for keeps\nShe won't walk away unless she knows she absolutely has to leave\n")
print(result[0]['generated_text'])

When given a song title, you will write a song in the style of Taylor Swift.
Title: When Emma Falls in Love
Lyrics: When Emma falls in love, she paces the floor
Closes the blinds and locks the door
When Emma falls in love, she calls up her mom
Jokes about the ways that this one could go wrong
She waits and takes her time
'Cause Little Miss Sunshine always thinks it's gonna rain
When Emma falls in love, I know
That boy will never be the same
'Cause she's the kind of book that you can't put down
Like if Cleopatra grew up in a small town
And all the bad boys would be good boys
If they only had a chance to love her
And to tell you the truth, sometimes I wish I was her
When Emma falls in love, it's all on her face
Hangs in the air like stars in outer space
When Emma falls in love, she disappears
And we all just laugh after seein' it all these years
When Emma falls apart, it's when she's alone
She takes on the pain and bears it on her own
'Cause when Emma falls in love, she's in it for keeps

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500, temperature=1.5)
result = pipe(f"When given a song title, you will write a song in the style of Taylor Swift.\nTitle: Treacherous (Taylor's Version)\nLyrics: Put your lips close to mine\nAs long as they don't touch\nOut of focus, eye to eye\n'Til the gravity's too much\nAnd I'll do anything you say\nIf you say it with your hands\nAnd I'd be smart to walk away\nBut you're quicksand\nThis slope is treacherous\nThis path is reckless\nThis slope is treacherous\nAnd I-I-I like it\n")
print(result[0]['generated_text'])


When given a song title, you will write a song in the style of Taylor Swift.
Title: Treacherous (Taylor's Version)
Lyrics: Put your lips close to mine
As long as they don't touch
Out of focus, eye to eye
'Til the gravity's too much
And I'll do anything you say
If you say it with your hands
And I'd be smart to walk away
But you're quicksand
This slope is treacherous
This path is reckless
This slope is treacherous
And I-I-I like it
This path is reckless
I-I-I, like it
Like there's nothing I could do or stop this slide
I don't know why, I've never known this kind of trust, mmm
I couldn't stop myself, even if I tried
And I'd be smarter to walk away
But the minute that I, ooh
Fell asleep at the wheel
I saw your face and
It's been a dark year, but this means so much to me
In the middle of winter
Anywhere with you
In this cold glow
The sun comes up, it is awakes that's slept in you for too long
I'll spend the whole weekend with you in New York, oh
This slope is treacherous
This path is reckle

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=600, temperature=1.5)
result = pipe(f"When given a song title, you will write a song in the style of Taylor Swift.\nTitle: Say Don't Go\nLyrics: I've known it from the very start\nWe’re a shot in the darkest dark\nOh no, oh no, I'm unarmed\nThe waiting is a sadness\nFading into madness\nOh no, oh no, it won't stop\nI’m standin' on a tightrope alone\nI hold my breath a little bit longer\nHalfway out the door, but it won't close\nI'm holdin' out hope for you to\nSay, \"Don't go\"\nI would stay forever if you say, \"Don't go\"\nWhy'd you have to lead me on?\nWhy’d you have to twist the knife?\nWalk away and leave me bleedin’, bleedin'?\nWhy’d you whisper in the dark?\nJust to leave me in the night?\nNow your silence has me screamin', screamin'\n(Say) Say, \"(Don't) don’t (Go) go\"\nI would stay forever if you (Say) say, \"(Don't) don't (Go) go\"\n(Say, say, say, say)\nNow I'm pacin' on shaky ground\nStrike a match, then you blow it out\nOh no, oh no, it's not fair\n'Cause you kiss me and it stops time\nAnd I'm yours, but you're not mine\nOh no, oh no, you're not there\n")
print(result[0]['generated_text'])

When given a song title, you will write a song in the style of Taylor Swift.
Title: Say Don't Go
Lyrics: I've known it from the very start
We’re a shot in the darkest dark
Oh no, oh no, I'm unarmed
The waiting is a sadness
Fading into madness
Oh no, oh no, it won't stop
I’m standin' on a tightrope alone
I hold my breath a little bit longer
Halfway out the door, but it won't close
I'm holdin' out hope for you to
Say, "Don't go"
I would stay forever if you say, "Don't go"
Why'd you have to lead me on?
Why’d you have to twist the knife?
Walk away and leave me bleedin’, bleedin'?
Why’d you whisper in the dark?
Just to leave me in the night?
Now your silence has me screamin', screamin'
(Say) Say, "(Don't) don’t (Go) go"
I would stay forever if you (Say) say, "(Don't) don't (Go) go"
(Say, say, say, say)
Now I'm pacin' on shaky ground
Strike a match, then you blow it out
Oh no, oh no, it's not fair
'Cause you kiss me and it stops time
And I'm yours, but you're not mine
Oh no, oh no, you're no

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700, temperature=1.5)
result = pipe(f"When given a song title, you will write a song in the style of Taylor Swift.\nTitle: Delicate\nLyrics: This ain't for the best\nMy reputation's never been worse, so\nYou must like me for me\nWe can't make\nAny promises now, can we, babe?\nBut you can make me a drink\nDive bar on the East Side, where you at?\nPhone lights up my nightstand in the black\nCome here, you can meet me in the back\nDark jeans and your Nikes, look at you\nOh, damn, never seen that color blue\nJust think of the fun things we could do\n('Cause I like you) This ain't for the best\nMy reputation's never been worse, so\nYou must like me for me\n(Yeah, I want you) We can't make\nAny promises now, can we, babe?\nBut you can make me a drink\nIs it cool that I said all that?\nIs it chill that you're in my head?\n'Cause I know that it's delicate (Delicate)\nIs it cool that I said all that?\nIs it too soon to do this yet?\n'Cause I know that it's delicate\nIsn't it? Isn't it? Isn't it?\nIsn't it?\nIsn't it? Isn't it? Isn't it?\nIsn't it delicate?\n")
print(result[0]['generated_text'])

When given a song title, you will write a song in the style of Taylor Swift.
Title: Delicate
Lyrics: This ain't for the best
My reputation's never been worse, so
You must like me for me
We can't make
Any promises now, can we, babe?
But you can make me a drink
Dive bar on the East Side, where you at?
Phone lights up my nightstand in the black
Come here, you can meet me in the back
Dark jeans and your Nikes, look at you
Oh, damn, never seen that color blue
Just think of the fun things we could do
('Cause I like you) This ain't for the best
My reputation's never been worse, so
You must like me for me
(Yeah, I want you) We can't make
Any promises now, can we, babe?
But you can make me a drink
Is it cool that I said all that?
Is it chill that you're in my head?
'Cause I know that it's delicate (Delicate)
Is it cool that I said all that?
Is it too soon to do this yet?
'Cause I know that it's delicate
Isn't it? Isn't it? Isn't it?
Isn't it?
Isn't it? Isn't it? Isn't it?
Isn't it delicate?
Can

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=800, temperature=1.5)
result = pipe(f"When given a song title, you will write a song in the style of Taylor Swift.\nTitle: Death By A Thousand Cuts\nLyrics: My, my, my, my\nMy, my, my, my\nMy, my, my, my\nMy, my, my, my\nMy, my, my, my\nMy, my, my, my\nMy, my, my, my\nMy, my, my, my\nSaying goodbye is death by a thousand cuts\nFlashbacks waking me up\nI get drunk, but it's not enough\n'Cause the morning comes and you're not my baby\nI look through the windows of this love\nEven though we boarded them up\nChandelier's still flickering here\n'Cause I can't pretend it's okay when it's not\nIt's death by a thousand cuts\nI dress to kill my time, I take the long way home\nI ask the traffic lights if it'll be alright\nThey say, \"I don't know\"\nAnd what once was ours is no one's now\nI see you everywhere, the only thing we share\nIs this small town\nYou said it was a great love, one for the ages\nBut if the story's over, why am I still writing pages?\n'Cause saying goodbye is death by a thousand cuts\nFlashbacks waking me up\nI get drunk, but it's not enough\n'Cause the morning comes and you're not my baby\nI look through the windows of this love\nEven though we boarded them up\nChandelier's still flickering here\n'Cause I can't pretend it's okay when it's not\nIt's death by a thousand cuts\nMy heart, my hips, my body, my love\nTryna find a part of me that you didn't touch\nGave up on me like I was a bad drug\nNow I'm searching for signs in a haunted club\n")
print(result[0]['generated_text'])

When given a song title, you will write a song in the style of Taylor Swift.
Title: Death By A Thousand Cuts
Lyrics: My, my, my, my
My, my, my, my
My, my, my, my
My, my, my, my
My, my, my, my
My, my, my, my
My, my, my, my
My, my, my, my
Saying goodbye is death by a thousand cuts
Flashbacks waking me up
I get drunk, but it's not enough
'Cause the morning comes and you're not my baby
I look through the windows of this love
Even though we boarded them up
Chandelier's still flickering here
'Cause I can't pretend it's okay when it's not
It's death by a thousand cuts
I dress to kill my time, I take the long way home
I ask the traffic lights if it'll be alright
They say, "I don't know"
And what once was ours is no one's now
I see you everywhere, the only thing we share
Is this small town
You said it was a great love, one for the ages
But if the story's over, why am I still writing pages?
'Cause saying goodbye is death by a thousand cuts
Flashbacks waking me up
I get drunk, but it's not enough

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500, temperature=1.5)
result = pipe(f"When given a song title, you will write a song in the style of Taylor Swift.\nTitle: mirrorball\nLyrics:")
print(result[0]['generated_text'])

When given a song title, you will write a song in the style of Taylor Swift.
Title: mirrorball
Lyrics: I still remember the games we played back then
How I always felt the same like the tumbleweeds
But the stars all lit up as if they knew
Back when this made no sense and I couldn't put it together
Didn't even know if any of this really happened
'Cause the plane takes off in clouds
And we land and I have your photograph
I still remember the house we held up when we were five
How your family taught me my favorite word to stay up late and live
But you saw right through me like I was paperface
And my world was comet tails after the Fourth of July fireworks
Back when this made no sense and I couldn't put it together
'Cause the plane takes off in clouds
And we land and I have your photograph
But, my God, this feels a lot like falling now
The past six months in particular have been surreal
And in case I ever forget again
Back when there wasn't proof
But the lights come on and I hear the voice

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500, temperature=1.5)
result = pipe(f"When given a song title, you will write a song in the style of Taylor Swift.\nTitle: gold rush\nLyrics: Gleaming, twinkling\nEyes like sinking ships on waters\nSo inviting, I almost jump in\nBut I don't like a gold rush, gold rush\nI don't like anticipatin' my face in a red flush\nI don't like that anyone would die to feel your touch\nEverybody wants you\nEverybody wonders what it would be like to love you\nWalk past, quick brush\nI don't like slow motion, double vision in rose blush\nI don't like that falling feels like flying 'til the bone crush\nEverybody wants you\nBut I don't like a gold rush\n")
print(result[0]['generated_text'])

When given a song title, you will write a song in the style of Taylor Swift.
Title: gold rush
Lyrics: Gleaming, twinkling
Eyes like sinking ships on waters
So inviting, I almost jump in
But I don't like a gold rush, gold rush
I don't like anticipatin' my face in a red flush
I don't like that anyone would die to feel your touch
Everybody wants you
Everybody wonders what it would be like to love you
Walk past, quick brush
I don't like slow motion, double vision in rose blush
I don't like that falling feels like flying 'til the bone crush
Everybody wants you
But I don't like a gold rush
No flash bulbs, they pass me
Everybody passes right through you and that makes two of us
No trendy dancin', whisper
When they tell you of my love that was the death of a dream
But it only made your neck and ears look so sweet
Everybody hates you
They only want to use you, fade you, but I was always too proud
Everybody loved you, they only want to abuse you, maim you
'Til you're mister and you miss Mr. Wrig

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500, temperature=1.5)
result = pipe(f"When given a song title, you will write a song in the style of Taylor Swift.\nTitle: Sweet Nothing\nLyrics: I spy with my little tired eye\nTiny as a firefly\nA pebble that we picked up last July\nDown deep inside your pocket\nWe almost forgot it\nDoes it ever miss Wicklow sometimes?\nOoh, ooh\nThey said the end is comin'\nEveryone's up to somethin'\nI find myself runnin' home to your sweet nothings\nOutside, they're push and shovin'\nYou're in the kitchen hummin'\nAll that you ever wanted from me was sweet nothin'\nOn the way home\nI wrote a poem\nYou say, \"What a mind\"\nThis happens all the time\nOoh, ooh\n'Cause they said the end is comin'\nEveryone's up to somethin'\nI find myself runnin' home to your sweet nothings\nOutside, they're push and shovin'\nYou're in the kitchen hummin'\nAll that you ever wanted from me was nothin'\n")
print(result[0]['generated_text'])

When given a song title, you will write a song in the style of Taylor Swift.
Title: Sweet Nothing
Lyrics: I spy with my little tired eye
Tiny as a firefly
A pebble that we picked up last July
Down deep inside your pocket
We almost forgot it
Does it ever miss Wicklow sometimes?
Ooh, ooh
They said the end is comin'
Everyone's up to somethin'
I find myself runnin' home to your sweet nothings
Outside, they're push and shovin'
You're in the kitchen hummin'
All that you ever wanted from me was sweet nothin'
On the way home
I wrote a poem
You say, "What a mind"
This happens all the time
Ooh, ooh
'Cause they said the end is comin'
Everyone's up to somethin'
I find myself runnin' home to your sweet nothings
Outside, they're push and shovin'
You're in the kitchen hummin'
All that you ever wanted from me was nothin'
Hey, we break up but
Babies mama heart is bad
Can't let 'em touch this
Honey, sweet daddyo
She only wants to give me
Sweet nothings, oh
They said the end is comin'
Everyone's up to so